# Application of amplitude estimation to Finances: Standard Approach Problems

In notebook *08_ApplicationTo_Finance_01_StandardApproach* we explain how to use **amplitude amplifications** techniques to compute expectation values and in the notebook: *09_ApplicationTo_Finance_02_Call_Option_BlackScholes* we use this for developing a complete price estimation for an *european call option* under the **Black Scholes** model.

In this notebook we are going to poitn out several problems of this standard approach and give some proposal solutions in order to avoid them.


In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm
from copy import deepcopy

In [ ]:
%matplotlib inline

In [ ]:
#This cell loads the QLM solver.
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg (for using QPU as CESGA QLM one)
from QQuantLib.utils.qlm_solver import get_qpu
QLMaaS = False
linalg_qpu = get_qpu(QLMaaS)

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from QQuantLib.utils.data_extracting import get_results

## 1. Amplitude Amplification and Expectation value computations.

As explained in notebook: *08_ApplicationTo_Finance_01_StandardApproach* the idea is compute the expectation of a function  $f(x)$ when $x$ follows proability density $p(x)$:

$$\mathbb{E}[f]=\int_a^bp(x)f(x)dx$$

that can be expresed as a Riemman sum:

$$\mathbb{E}[f] = \sum_{i=0}^{2^n-1} p(x_i)f(x_i)$$

using **amplitude amplification** algorithms. 

In this kind of algorithms we have a quantum state $|\Psi\rangle$ that can be decomposed in the following way:

$$|\Psi\rangle=\sqrt{a}|\Psi_{0}\rangle+\sqrt{1-a}|\Psi_{1}\rangle$$

where:
$$
\begin{array}{l}
&\sqrt{a}|\Psi_{0}\rangle = |0\rangle \otimes\sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)f(x_i)}|i\rangle_{n},\\\\
&\sqrt{1-a}|\Psi_{1}\rangle = |1\rangle\otimes\sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)(1-f(x_i))}|i\rangle_{n}.
\end{array}
$$
The probability of measuring $|0\rangle$ in the leftmost qubit is:

$$ a = \sum_{i=0}^{2^{n}-1}\left|\sqrt{p(x_i)f(x_i)}\right|^2,$$

## 2. Problems of the standard procedure

One of the major problems of the procedure summarize in Section 1 is the following:

If we have negative values in $f$, we won't be able to compute the correct sum because, when we measure the leftmost qubit we are taking the absolute value:

$$\sum_{i=0}^{2^{n}-1}\left|\sqrt{p(x_i)f(x_i)}\right|^2 = \sum_{i=0}^{2^{n}-1}p(x_i)\left|f(x_i)\right| $$

For demonstrating this we are going to develop following example:

* Domain: our $x$ will be a set of $2^{n}$ integers numbers.
* $p(x)$: Over our domain we are going to define a properly normalised density distribution in the form:
$$p(x)=\frac{x}{\sum_{i=0}^{2^{n}-1}i}$$

we are going to use $n=3$ and the following function $f(x)$:

$$f = \dfrac{1}{7}\left(-0,-1,-2,-3,4,5,6,7\right).$$


In [ ]:
n = 3
N = 2**n
x = np.arange(N)

p_X = x/np.sum(x)

f_X = np.copy(p_X)
f_X[1] = -p_X[1]
f_X[2] = -p_X[2]
f_X[3] = -p_X[3]
print(f_X)

Now we load the data using the usual way

In [ ]:
from QQuantLib.DL.data_loading import load_probability, load_array, uniform_distribution

In [ ]:
p_gate = load_probability(p_X)
#For avoiding problems we need to provide absolute values of f
f_gate = load_array(np.sqrt(np.abs(f_X)))


oracle_problem = qlm.QRoutine()
register_problem = oracle_problem.new_wires(n+1)
oracle_problem.apply(p_gate, register_problem[:n])
oracle_problem.apply(f_gate, register_problem)

%qatdisplay oracle_problem

In [ ]:
print('p(x) condition: {}'.format(np.isclose(np.sum(p_X), 1)))
print('f(x) condition: {}'.format(np.max(f_X) <= 1))

In [ ]:
print("Exact solution: ",np.dot(p_X,f_X))

Now we can use the **amplitude estimation** algorithms for calculating the desired integral

In [ ]:
target = [0]
index = [oracle_problem.arity-1]

In [ ]:
from QQuantLib.AE.maximum_likelihood_ae import MLAE

mlae_dict = {
    'qpu': linalg_qpu,
}
mlae = MLAE(
    oracle_problem,
    target = target,
    index = index, 
    **mlae_dict
)

mlae_a = mlae.run()
print('mlae_a: ', mlae_a)

In [ ]:
from QQuantLib.PE.phase_estimation_wqft import PE_QFT_AE

ae_pe_qft_dict = {
    'qpu': linalg_qpu,
    'auxiliar_qbits_number': 8,
    'shots': 100
}

ae_pe_qft = PE_QFT_AE(
    oracle_problem,
    target = target,
    index = index, 
    **ae_pe_qft_dict
)
ae_pe_qft_a  = ae_pe_qft.run()

print('ae_pe_qft_a: ', ae_pe_qft_a)

In [ ]:
from QQuantLib.PE.iterative_quantum_pe import IQPE_AE

ae_iqpe_dict = {
    'qpu': linalg_qpu,
    'cbits_number': 8,
    'shots': 10
}

ae_iqpe = IQPE_AE(
    oracle_problem,
    target = target,
    index = index, 
    **ae_iqpe_dict
)

ae_iqpe_a  = ae_iqpe.run()

print('ae_iqpe_a: ', ae_iqpe_a)

In [ ]:
from QQuantLib.AE.iterative_quantum_ae import IQAE

iqae_dict = {
    'qpu': linalg_qpu
}

iqae = IQAE(
    oracle_problem,
    target = target,
    index = index, 
    **iqae_dict
)

iqae_a = iqae.run()

print('iqae_a: ', iqae_a)

In [ ]:
methods = ['MLAE', 'PE_QFT_AE', 'IQPE_AE', 'IQAE']
a_estimated = [mlae.a, ae_pe_qft.a, ae_iqpe.a, iqae.a]


dic_staff = {
    'AE_a': a_estimated,
}

Results = pd.DataFrame(dic_staff, index=methods)
Results['Exact_Solution'] = np.dot(p_X,f_X)

In [ ]:
Results

As can be seen none of all the used methods give us the correct answer!

## 3. New Loading Data Procedure

The reason for the fail showed in Section 2 is our loading data procedure. If we review the notebook: *01_Data_Loading_Module_Use* we see that our *load_probability*  function from the **DL/data_loading** module loads the $\sqrt{p(x)}$. Additionally when we load the function $f(x)$ we, really, load the $\sqrt{f(x)}$. 

On easy workarond for dealing with section 2 problem is changing our data loading procedure in the following ways:
* We are going to load $p(x)$ and $f(x)$ instead of $\sqrt{p(x)}$ and $\sqrt{f(x)}$
* The probability distribution will be loaded as a function insted of using the probability density procedure.

If we have discretized the $p(X)$ and the $f(x)$ functions in $2^{n}$ values this new loading procedure will need $n+2$ qbits instead of the $n+1$ qbit of the original procedure.

Here we resumen the new loading data protocol:

1. We begin with a $n+2$ qbits state (the 1 and 2 superindices is for indetifiyng the qbit)

$$|0\rangle^1 \otimes |0\rangle^2 \otimes|0\rangle_{n}$$

2. We apply uniform distribution over the $n$ qbits state (the 1 and 2 sub-indices are for indicating over each qbit the operator should be applied)

$$(I_1 \otimes I_2 \otimes H^{\otimes n})|0\rangle^1 \otimes |0\rangle^2 \otimes|0\rangle_{n} = |0\rangle^1 \otimes |0\rangle^2 \otimes H^{\otimes n}|0\rangle_{n}=|0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}|i\rangle_{n}$$

3. We apply the **loading function operator** for loading the probability distribution $p(X)$: $\mathcal{F(p)}$ over the qbit $|0\rangle^2$

$$\big(I_1 \otimes \mathcal{F_2(p)}\big) \big(|0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}|i\rangle_{n} \big) = |0\rangle^1 \otimes \mathcal{F_2(p)} \big( |0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}|i\rangle_{n} \big) $$

$$=|0\rangle^1 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}|i\rangle_{n} \otimes [ p(i) |0\rangle^2 +  \sin(\theta_{p(i)}) |1\rangle^2] =$$


$$=|0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}p(i)|i\rangle_{n} +  |0\rangle^1 \otimes |1\rangle^2 \otimes ...$$

4. In the last expresion we only are interested in the terms with $|0\rangle^1 \otimes |0\rangle^2$ the other option does not interest us, so in the following steps we delete from the formulas. So the importan part will be:

$$|0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}p(i)|i\rangle_{n}$$

5. Now we apply the **loading function operator** for loading $f(x)$: $\mathcal{F(f)}$ over the qbit $|0\rangle^1$

$$\big(I_2 \otimes \mathcal{F_1(f)}\big) \big(|0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}p(i)|i\rangle_{n} \big)= |0\rangle^2 \otimes \mathcal{F_1(f)} \big(|0\rangle^1 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}p(i)|i\rangle_{n}\big)=$$


$$=|0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}p(i)|i\rangle_{n} \otimes [ f(i) |0\rangle^1 +  \sin(\theta_{f(i)}) |1\rangle^1] = $$

$$=|0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}p(i)f(i)|i\rangle_{n} + |1\rangle^1 \otimes |0\rangle^2 \otimes ...$$

6. Again we are interested only on the therms with $|0\rangle^1 \otimes |0\rangle^2$ so we take of from the formulas the other terms. So the important part will be:

$$|0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}p(i)f(i)|i\rangle_{n}$$

7. Finally we apply another uniform distribution over the $n$ qbits state:

$$\big(I_1 \otimes I_2 \otimes H^{\otimes n}\big) \big(|0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}p(i)f(i)|i\rangle_{n}\big) = |0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}p(i)f(i)H^{\otimes n}|i\rangle_{n}$$


The final state of our new loading protocol will be in the form:

$$|\Psi \rangle = |0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{\sqrt{2^n}} \sum_{i=0}^{2^{n}-1}p(i)f(i)H^{\otimes n}|i\rangle_{n} + |0\rangle^1 \otimes |1\rangle^2 \otimes ... + |1\rangle^1 \otimes |0\rangle^2 \otimes ... + |1\rangle^1 \otimes |1\rangle^2 \otimes ...$$

The uniform distribution acting over any $|i\rangle_n$ state can be expresed as:

$$H^{\otimes n}|i\rangle_{n} = \frac{1}{\sqrt{2^n}}\sum_{j=0}^{2^{n}-1} (-1)^{ij}|j\rangle_n=\frac{1}{\sqrt{2^n}}[|0\rangle_n + \sum_{j=1}^{2^{n}-1} (-1)^{ij}|j\rangle_n]$$ 

So replacing it in our final state:

$$|\Psi \rangle = |0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{2^n} \sum_{i=0}^{2^{n}-1}p(i)f(i)|0\rangle_n + |0\rangle^1 \otimes |0\rangle^2 \otimes \frac{1}{2^n}\sum_{j=1}^{2^{n}-1} (-1)^{ij}p(i)f(i)|j\rangle_n + |0\rangle^1 \otimes |1\rangle^2 \otimes ... + |1\rangle^1 \otimes |0\rangle^2 \otimes ... + |1\rangle^1 \otimes |1\rangle^2 \otimes ...$$


As can be seen the term that carries the information we need is: $|0\rangle^1 \otimes |0\rangle^2 \otimes|0\rangle_n$. We do not care from other terms so we can write the final state in the following way:


$$|\Psi \rangle = \frac{1}{2^n}\sum_{i=0}^{2^{n}-1}p(i)f(i)|0\rangle^1 \otimes |0\rangle^2 \otimes|0\rangle_n+...$$

All this procedure is implemented in the following cell:

In [ ]:
new_oracle = qlm.QRoutine()
#For new data loading procedure we need n+2 qbits
new_registers = new_oracle.new_wires(n+2)
#Step 2 of Procedure: apply Uniform distribution 
new_oracle.apply(uniform_distribution(n),new_registers[:n])
#Step 3 of Procedure: apply loading function operator for loading p(x)
new_p_gate = load_array(p_X, id_name = 'p(x)')
new_oracle.apply(new_p_gate, [new_registers[:n], new_registers[n]])
#Step 5 of Procedure: apply loading function operator for loading f(x)
new_f_gate = load_array(f_X, id_name = 'f(x)')
new_oracle.apply(new_f_gate, [new_registers[:n], new_registers[n+1]])
#Step 7 of Procedure: apply Uniform distribution 
new_oracle.apply(uniform_distribution(n),new_registers[:n])

In [ ]:
%qatdisplay new_oracle

Now with the new loading procedure we can again express the quantum state as a linear combination of two orthogonal states $|\Psi_{1}\rangle$ y $|\Psi_{0}\rangle$:

$$|\Psi\rangle = \sqrt{a}|\Psi_{0}\rangle+\sqrt{1-a}|\Psi_{1}\rangle$$

where:
$$\sqrt{a}|\Psi_{0}\rangle = \left[\dfrac{1}{2^n}\sum_{i=0}^{2^{n}-1}p(i)f(i)\right]|0\rangle^1 \otimes|0\rangle^2 \otimes|0\rangle_n$$

Now we can remove the super-index from 1 qbit $|0\rangle$ state:

$$\sqrt{a}|\Psi_{0}\rangle = \left[\dfrac{1}{2^n}\sum_{i=0}^{2^{n}-1}p(i)f(i)\right]|0\rangle \otimes|0\rangle \otimes|0\rangle_n$$
 

The probability of measuring $|0\rangle \otimes|0\rangle \otimes|0\rangle_n$ is:

$$ a= \left|\dfrac{1}{N}\sum_{i=0}^{2^{n}-1}p(i)f(i)\right|^2,$$

so then computing $\sqrt{a}$ gives us an estimation.

Now we can use again the *amplitude estimation* routines, but using the right target and index!!

In [ ]:
new_target = [0 for i in range(new_oracle.arity)]
print('new_target: ', new_target)
new_index = [i for i in range(new_oracle.arity)]
print('new_index: ', new_index)

In [ ]:
from QQuantLib.AE.maximum_likelihood_ae import MLAE

m_k = [i for i in range(30)]
n_k = [100 for i in m_k]


mlae_dict = {
    'qpu': linalg_qpu,
    'schedule': [m_k, n_k]
}
mlae = MLAE(
    new_oracle,
    target = new_target,
    index = new_index, 
    **mlae_dict
)

mlae_a = mlae.run()
print('mlae_a: ', mlae_a)

In [ ]:
from QQuantLib.PE.phase_estimation_wqft import PE_QFT_AE

ae_pe_qft_dict = {
    'qpu': linalg_qpu,
    'auxiliar_qbits_number': 10,
    'shots': 100
}

ae_pe_qft = PE_QFT_AE(
    new_oracle,
    target = new_target,
    index = new_index, 
    **ae_pe_qft_dict
)
ae_pe_qft_a  = ae_pe_qft.run()

print('ae_pe_qft_a: ', ae_pe_qft_a)

In [ ]:
from QQuantLib.PE.iterative_quantum_pe import IQPE_AE

ae_iqpe_dict = {
    'qpu': linalg_qpu,
    'cbits_number': 10,
    'shots': 10
}

ae_iqpe = IQPE_AE(
    new_oracle,
    target = new_target,
    index = new_index, 
    **ae_iqpe_dict
)

ae_iqpe_a  = ae_iqpe.run()

print('ae_iqpe_a: ', ae_iqpe_a)

In [ ]:
from QQuantLib.AE.iterative_quantum_ae import IQAE

epsilon = 0.001
iqae_dict = {
    'qpu': linalg_qpu,
    'epsilon': epsilon
}

iqae = IQAE(
    new_oracle,
    target = new_target,
    index = new_index,  
    **iqae_dict
)

iqae_a = iqae.run()

print('iqae_a: ', iqae_a)

**BE AWARE**

Now the **amplitude estimation** routines will estimate:

$$ a= \left|\dfrac{1}{2^n}\sum_{i=0}^{2^{n}-1}p(i)f(i)\right|^2$$

so if we measured $a$ for getting the proper result we need:

$$\sum_{i=0}^{2^{n}-1}p(i)f(i) = 2^n * \sqrt{a}$$

So in order to get the orrect a we must provide the $\sqrt{a}$

In [ ]:
methods = ['MLAE', 'PE_QFT_AE', 'IQPE_AE', 'IQAE']
a_estimated = [mlae.a, ae_pe_qft.a, ae_iqpe.a, iqae.a]


dic_staff = {
    'AE_a': np.sqrt(a_estimated)*2**n,
}

new_Results = pd.DataFrame(dic_staff, index=methods)
new_Results['Exact_Solution'] = np.dot(p_X,f_X)

In [ ]:
new_Results

## 5. Problems of the first proposal of solution

We have solved one of the issue, but we still face one major problem. If the expectation of $f$ is negative, we won't be able to notice it, again because we are taking the absolute value.
To demonstrate it let us define a new linear function $f$ which is negative in the second half:
$$f = \dfrac{1}{7}\left(0,1,2,3,-4,-5,-6,-7\right).$$

In [ ]:
f3 = np.copy(f_X)
f3[4] = -f_X[4]
f3[5] = -f_X[5]
f3[6] = -f_X[6]
f3[7] = -f_X[7]

In [ ]:
p4_gate = load_array(p_X)
f4_gate = load_array(f3,id_name = "2")

In [ ]:
oracle4 = qlm.QRoutine()
register4 = oracle4.new_wires(n+2)
oracle4.apply(uniform_distribution(n),register4[:n])
oracle4.apply(p4_gate, [register4[:n],register4[n]])
oracle4.apply(f4_gate, [register4[:n],register4[n+1]])
oracle4.apply(uniform_distribution(n),register4[:n])

In [ ]:
target = [0,0,0,0,0]
index = [0,1,2,3,4]
iqae = IQAE(oracle = oracle4,target = target, index = index)

In [ ]:
epsilon = 0.001
N = 556
alpha = 0.05 
bounds = iqae.run(epsilon = epsilon,N = N,alpha = alpha)

In [ ]:
print("Estimation: ",2**n*np.sqrt(bounds[0]),2**n*np.sqrt(bounds[1]))
print("Exact solution: ",np.dot(p_X,f3))

Here we see that the module is correct but not the sign. Of course this becomes critical as it is not the same obtining an expected positve result or a negative one. For instance, think of the return on an investment.

In [ ]:
iqae.display_information(epsilon = epsilon,N = N,alpha = alpha)

## 6. A more definitive approach

The solution to this is using an algorithm which distinguishes the sign of the underlying amplitude. For that we propose the RQAE, for more information check the corresponding notebook and article.

In [ ]:
from QQuantLib.AE.real_quantum_ae import RQAE

In [ ]:
target = [0]*(n+2)
index = np.arange(n+2)
rqae = RQAE(oracle = oracle4,target = target, index = index)

In [ ]:
q = 2
epsilon = 0.001
gamma = 0.05 
bounds = rqae.run(q = q, epsilon = epsilon,gamma = gamma)

In [ ]:
print("Estimation: ",2**n*bounds[0],2**n*bounds[1])
print("Exact solution: ",np.dot(p_X,f3))

In [ ]:
rqae.display_information(q = q, epsilon = epsilon,gamma = gamma)

## 7. Call option under the Black-Scholes model

Now we will use this same strategy to compute the prices of a plain vanilla under the assumptions of the Black-Scholes model. In order to do that, we have to choose some parameters for the pricing model, in this case:
- Current underlying price ($S_0$): 2
- Risk-free rate ($r$): 4\%
- The volatility ($\sigma$): 10\%

Next, we define the parameters of the call option:
- Maturity ($T$): 300 days 
- Strike ($K$): 1.9

In [ ]:
S0 = 2
r = 0.04
sigma = 0.1
T = 300/365
K = 1.9

The price of a call option can be approximated as:
$$C(S_0,K,T) = e^{-rT}\mathbb{E}_r[(S_T-K)^+]$$
In the next cell we compute the approximated value when we approximate the probability distribution and the payoff with $2^n$ points.

In [ ]:
from QQuantLib.utils.utils import bs_probability, call_payoff

In [ ]:
n = 3
x = np.linspace(1,3,2**n)
probability = bs_probability(x,S0,r,sigma,T)
payoff = call_payoff(x,K)
classical_price_estimation = np.exp(-r*T)*np.dot(probability,payoff)

Now we will compute the approximated price via the quantum computer. Before we load the probability and the payoff into the quantum computer we have to normalise the payoff function.

In [ ]:
payoff_normalisation = np.max(payoff)
payoff_normalised = payoff/payoff_normalisation

In [ ]:
p5_gate = load_array(probability,id_name = "probability")
f5_gate = load_array(payoff_normalised,id_name = "payoff")

In [ ]:
oracle5 = qlm.QRoutine()
register5 = oracle5.new_wires(n+2)
oracle5.apply(uniform_distribution(n),register5[:n])
oracle5.apply(p5_gate, [register5[:n],register5[n]])
oracle5.apply(f5_gate, [register5[:n],register5[n+1]])
oracle5.apply(uniform_distribution(n),register5[:n])

In [ ]:
target = [0]*(n+2)
index = np.arange(n+2)
rqae = RQAE(oracle = oracle5,target = target, index = index)

In [ ]:
q = 2
epsilon = 0.001
gamma = 0.05 
bounds_rqae = rqae.run(q = q, epsilon = epsilon,gamma = gamma)

In [ ]:
quantum_price_estimation = (bounds_rqae[0]+bounds_rqae[1])/2*2**n*payoff_normalisation*np.exp(-r*T)

Last, we use function *bs_call_price* to compute the exact price and compare it with the classical and quantum estimation

In [ ]:
from QQuantLib.utils.utils import bs_call_price

In [ ]:
exact_price = bs_call_price(S0,r,sigma,T,K)

In [ ]:
print("Exact price: ",exact_price)
print("Classical estimation: ",classical_price_estimation)
print("Quantum estimation: ",quantum_price_estimation)

Note that this procedure does not provide any quantum advantage.

## 8. Futures payoff under Black-Scholes model 

To emphasize the advantages of the RQAE method we will use the same strategy to compute the prices of a derivatives contract with linear payoff under the assumptions of the Black-Scholes model. In order to do that, we have to choose some parameters for the pricing model, in this case:
- Current underlying price ($S_0$): 2
- Risk-free rate ($r$): 4\%
- The volatility ($\sigma$): 10\%

Next, we define the parameters of the derivatives contract:
- Maturity ($T$): 300 days 
- Strike ($K$): 2.5

In [ ]:
S0 = 2
r = 0.04
sigma = 0.1
T = 300/365
K = 2.5

The price of this derivatives contract with linear payoff can be approximated:
$$C(S_0,K,T) = e^{-rT}\mathbb{E}_r[S_T-K]$$
In the next cell we compute the approximated value when we approximate the probability distribution and the payoff with $2^n$ points.

In [ ]:
n = 3
x = np.linspace(1,3,2**n)
probability = bs_probability(x,S0,r,sigma,T)
payoff = x-K
classical_price_estimation = np.exp(-r*T)*np.dot(probability,payoff)
print("Classical price estimation: ",classical_price_estimation)

As we see the price of this contract is negative. As we saw before, the standard quantum procedure won't work so we use RQAE. With it we compute the approximated price via the quantum computer. Before we load the probability and the payoff into the quantum computer we have to normalise the payoff function.

In [ ]:
payoff_normalisation = np.max(np.abs(payoff))
payoff_normalised = payoff/payoff_normalisation

In [ ]:
p5_gate = load_array(probability,id_name = "probability")
f5_gate = load_array(payoff_normalised,id_name = "payoff")

In [ ]:
oracle5 = qlm.QRoutine()
register5 = oracle5.new_wires(n+2)
oracle5.apply(uniform_distribution(n),register5[:n])
oracle5.apply(p5_gate, [register5[:n],register5[n]])
oracle5.apply(f5_gate, [register5[:n],register5[n+1]])
oracle5.apply(uniform_distribution(n),register5[:n])

In [ ]:
target = [0]*(n+2)
index = np.arange(n+2)
rqae = RQAE(oracle = oracle5,target = target, index = index)

In [ ]:
q = 2
epsilon = 0.001
gamma = 0.05 
bounds_rqae = rqae.run(q = q, epsilon = epsilon,gamma = gamma)

In [ ]:
quantum_price_estimation = (bounds_rqae[0]+bounds_rqae[1])/2*2**n*payoff_normalisation*np.exp(-r*T)

Last, we compare the classical and quantum estimation:

In [ ]:
print("Classical estimation: ",classical_price_estimation)
print("Quantum estimation: ",quantum_price_estimation)

## 8. Computation of VaR

For a given confidence level $\alpha$, $VaR_\alpha(X)$ is the smallest value $x$ such that 
$$P[X\leq x]\geq (1-\alpha).$$ To compute this quantity we can do a binary search.

### 8.1 Classical binary search

A binary search for $N = 2^n$ discrete probabilities works as follows:
- We begin by computing the cumulative sum for the first $2^{n-1}$ probabilities.
- If the probability is lower than the one we demand, we set $2^{n-1} = 2^{n-1}+2^{n-2}$ entries. Otherwise, we set $2^{n-1} = 2^{n-1}-2^{n-2}$ entries.
- Now we can start in the first step until $N = 1$, then we stop.
In the following cells we do this process in an iterative fashion to compute the VaR

We will do this process with the probability distribution from Black-Scholes equation

In [ ]:
probability = bs_probability(x,S0,r,sigma,T)
#probability = np.array([0,0,0,0,0.96,0.04,0,0])

In [ ]:
alpha = 0.05
index = 0
sign = 1
for i in range(1,n+1):
    index = index+sign*2**(n-i)
    suma = np.sum(probability[0:index])
    if (suma<=1-alpha):
        sign = 1
    else:
        sign = -1
if (sign==1): 
    index = index+sign

print("El resultado es: ",index)

In [ ]:
np.sum(probability[:index])

### 8.2 Quantum binary search with IQAE

The quantum binary search using IQAE proceeds in the same manner as the classical binary search. The main difference is in the quanutm way we perform the cumulative sum. For that purpose we have defined the step function $s_i^N$. This function flips the leftmost qubit of all the states $j$ with the condition $j\geq i$. 

Let us give an example with $3+1$ qubits. We start with the state:
$$|\phi\rangle_0 = |0\rangle\otimes\left[\sqrt{p_0}|0\rangle+\sqrt{p_1}|1\rangle+\sqrt{p_2}|2\rangle+\sqrt{p_3}|3\rangle+\sqrt{p_4}|4\rangle+\sqrt{p_5}|5\rangle+\sqrt{p_6}|6\rangle+\sqrt{p_7}|7\rangle\right].$$
An application of the function $s_4^8$ yields:
$$s_4^8|\phi\rangle_0 = |0\rangle\otimes\left[\sqrt{p_0}|0\rangle+\sqrt{p_1}|1\rangle+\sqrt{p_2}|2\rangle+\sqrt{p_3}|3\rangle\right]+|1\rangle\otimes\left[\sqrt{p_4}|4\rangle+\sqrt{p_5}|5\rangle+\sqrt{p_6}|6\rangle+\sqrt{p_7}|7\rangle\right].$$
Now, performing the sum of the first four entries is equivalent to measuring the probability of obtaining $|0\rangle$ in the leftmost qubit. This is the strategy to perform the cumulative sums.

Note that, in the cell where we perform the binary search we only substitute the line of the cumulative sum with this new strategy, the rest remains the same.


In [ ]:
from QQuantLib.DL.data_loading import step_array

In [ ]:
probability = bs_probability(x,S0,r,sigma,T)
#probability = np.array([0,0,0,0,0.96,0.04,0,0])
p_gate = load_probability(probability)

In [ ]:
N = 100
epsilon = 0.01

alpha = 0.05
index = 0
sign = 1
for i in range(1,n+1):
    index = index+sign*2**(n-i)
    # Sum process done in the quantum computer
    routine = qlm.QRoutine()
    register = routine.new_wires(n+1)
    routine.apply(p_gate,register[:n])
    routine.apply(step_array(index,2**n),register)
    ##########################################
    iqae = IQAE(oracle = routine,target = [0], index = [n])
    bounds = iqae.run(epsilon = epsilon,N = N,alpha = 0.05)
    suma = (bounds[0]+bounds[1])/2
    ##########################################
    if (suma<=1-alpha):
        sign = 1
    else:
        sign = -1
if (sign==1): 
    index = index+sign

print("El resultado es: ",index)

In [ ]:
np.sum(probability[:index])

### 8.3 Quantum binary search with RQAE

Once again, the quantum binary search with RQAE proceeds in the same manner as the classical binary search. The main difference is in the quanutm way we perform the cumulative sum. As here we work with the amplitudes instead of the probabilities we have to perform the sum in the quantum state.

Let us give an example with $3+1$ qubits. We start with the state:
$$|\phi\rangle_0 = \dfrac{1}{\sqrt{2^3}}|0\rangle\otimes |0\rangle\otimes\left[p_0|0\rangle+p_1|1\rangle+p_2|2\rangle+p_3|3\rangle+p_4|4\rangle+p_5|5\rangle+p_6|6\rangle+p_7|7\rangle\right]+ ....$$
An application of the step function $s_4^8$ yields:
$$s_4^8|\phi\rangle_0 =  \dfrac{1}{\sqrt{2^3}}|0\rangle\otimes|0\rangle\otimes\left[p_0|0\rangle+p_1|1\rangle+p_2|2\rangle+p_3|3\rangle\right]+|1\rangle\otimes|0\rangle\otimes\left[p_4|4\rangle+p_5|5\rangle+p_6|6\rangle+p_7|7\rangle\right]+....$$
Now, performing the sum of the first four entries is equivalent to applying Hadamard gates to the first entries:
$$H^{\otimes3}|\phi\rangle_1 =  \dfrac{p_0+p_1+p_2+p_3}{2^3}|0\rangle\otimes|0\rangle\otimes|0\rangle+...$$
This is the strategy to perform the cumulative sums.

Note that, in the cell where we perform the binary search we only substitute the line of the cumulative sum with this new strategy, the rest remains the same.



In [ ]:
probability = bs_probability(x,S0,r,sigma,T)
#probability = np.array([0,0,0,0,0.96,0.04,0,0])
p_gate = load_array(probability,id_name = "probability")

In [ ]:
gamma = 0.05
q = 2
epsilon = 0.01

alpha = 0.05
index = 0
sign = 1
for i in range(1,n+1):
    index = index+sign*2**(n-i)
    # Sum process done in the quantum computer
    routine = qlm.QRoutine()
    register = routine.new_wires(n+2)
    routine.apply(uniform_distribution(n),register[:n])
    routine.apply(p_gate,[register[:n],register[n]])
    routine.apply(step_array(index,2**n),[register[0:n],register[n+1]])
    routine.apply(uniform_distribution(n),register[:n])
    ##########################################
    rqae = RQAE(oracle = routine,target = [0]*(n+2), index = np.arange(n+2))
    bounds = rqae.run(q = q,epsilon = epsilon,gamma = gamma)
    suma = np.abs(bounds[1]*2**n)
    ##########################################
    if (suma<=1-alpha):
        sign = 1
    else:
        sign = -1
if (sign==1): 
    index = index+sign

print("El resultado es: ",index)

In [ ]:
np.sum(probability[:index])